# Notebook 03 — Wikipedia Circuits Exploration  

## 1. Présentation de la source : Wikipedia

Wikipedia est utilisée comme source de **référencement géographique** des
circuits de F1, fournissant nom, localisation, ville/pays et coordonnées
(latitude/longitude). Ces données servent de socle pour :
- contextualiser les données Meteostat,  
- sélectionner les stations météo pertinentes,  
- enrichir les données OpenF1 en Silver.  

**Positionnement Bronze** :  
- données brutes, non enrichies, sans jointure ni correction,  
- traçabilité explicite (source, date de scraping),  
- cohérence avec le rôle exploratoire de la couche Bronze.  

**Limites et posture critique** :  
- maintenance collaborative, mises à jour non garanties,  
- possibles incohérences de format ou granularité.  

Ces limites sont documentées, l’usage se limite aux données de référence
statiques, sans exploitation de données critiques ou dynamiques.

**Conclusion** : Wikipedia fournit une référence géographique fiable et
traçable, prête pour un enrichissement ultérieur en Silver.

---------

## 2. Méthodologie de scraping

Objectif : extraire de manière claire, reproductible et tracée les données
géographiques des circuits, en respectant la **couche Bronze**.

Stratégie :  
1. **Page de référence** : identification de tous les circuits de F1.  
2. **Pages individuelles** : extraction des infos géographiques (latitude, longitude) depuis les infobox.

Points clés :  
- données brutes conservées, valeurs manquantes acceptées, pas de correction  
- approche robuste, limitée aux pages dédiées pour gérer variations et
incohérences  
- extraction en Python avec `requests`, `BeautifulSoup`, `pandas`  
- headers explicites pour bonnes pratiques et traçabilité  

Gestion des erreurs : pages indisponibles, infobox partielles ou champs
manquants sont documentés et conservés en **Bronze** pour analyse qualité ultérieure.

-------------

## 3. Scrapping

### Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from pathlib import Path

### Initialisation du scrapping

In [ ]:
HEADERS = {
    "User-Agent": "F1PredictiveAssistant/1.0 (Academic project, data exploration)"
}

In [3]:
WIKIPEDIA_CIRCUITS_URL = "https://en.wikipedia.org/wiki/List_of_Formula_One_circuits"

In [4]:
response = requests.get(WIKIPEDIA_CIRCUITS_URL, headers=HEADERS)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

In [5]:
soup.title.text

'List of Formula One circuits - Wikipedia'

### Extraction des infos circuits

In [6]:
tables = soup.find_all("table", {"class": "wikitable"})
print(f"Nombre de tables trouvées : {len(tables)}")

Nombre de tables trouvées : 2


La page Wikipedia contient deux tables de type `wikitable`.

- une légende (circuits actuels / futurs).
- *"Formula One circuits"* qui contient la liste complète des circuits utilisés en F1.

Seule la seconde est pertinente pour notre extraction.

In [7]:
circuits_table = tables[1]

rows = circuits_table.find("tbody").find_all("tr")[1:]

print(f"Nombre de lignes (circuits) trouvées : {len(rows)}")

Nombre de lignes (circuits) trouvées : 78


### Données extraites de la table principale

- Nom du circuit  
- Lien Wikipedia  
- Ville / localité  
- Pays  

Ces informations sont déjà séparées, évitant le parsing de chaînes composites.
La latitude et longitude ne sont pas encore récupérées à ce stade.

In [ ]:
circuits_list = []

BASE_WIKI_URL = "https://en.wikipedia.org"

for row in rows:
    cols = row.find_all("td")
    if len(cols) < 6:
        continue

    circuit_link = cols[0].find("a")
    circuit_name = circuit_link.get_text(strip=True) if circuit_link else None
    circuit_url = BASE_WIKI_URL + circuit_link["href"] if circuit_link else None

    location_links = cols[4].find_all("a")
    locality = location_links[0].get_text(strip=True) if location_links else None

    country_links = cols[5].find_all("a")
    country = country_links[-1].get_text(strip=True) if country_links else None

    circuits_list.append({
        "circuit_name": circuit_name,
        "circuit_url": circuit_url,
        "locality": locality,
        "country": country
    })

print(f"Circuits extraits depuis la liste : {len(circuits_list)}")
circuits_list[:3]

Circuits extraits depuis la liste : 78


[{'circuit_name': 'Adelaide Street Circuit',
  'circuit_url': 'https://en.wikipedia.org/wiki/Adelaide_Street_Circuit',
  'locality': 'Adelaide',
  'country': 'Australia'},
 {'circuit_name': 'Ain-Diab Circuit',
  'circuit_url': 'https://en.wikipedia.org/wiki/Ain-Diab_Circuit',
  'locality': 'Casablanca',
  'country': 'Morocco'},
 {'circuit_name': 'Aintree Motor Racing Circuit',
  'circuit_url': 'https://en.wikipedia.org/wiki/Aintree_Motor_Racing_Circuit',
  'locality': 'Aintree',
  'country': 'United Kingdom'}]

### Structuration tabulaire

La liste des circuits, initialement sous forme de dictionnaires Python, est
convertie en **DataFrame pandas** pour faciliter l’enrichissement, l’analyse de
qualité et le stockage en **couche Bronze**. Les données extraites restent inchangées.

In [9]:
circuits_df = pd.DataFrame(circuits_list)

circuits_df

,circuit_name,circuit_url,locality,country
0,Adelaide Street Circuit,https://en.wikipedia.org/wiki/Adelaide_Street_...,Adelaide,Australia
1,Ain-Diab Circuit,https://en.wikipedia.org/wiki/Ain-Diab_Circuit,Casablanca,Morocco
2,Aintree Motor Racing Circuit,https://en.wikipedia.org/wiki/Aintree_Motor_Ra...,Aintree,United Kingdom
3,Albert Park Circuit,https://en.wikipedia.org/wiki/Albert_Park_Circuit,Melbourne,Australia
4,Algarve International Circuit,https://en.wikipedia.org/wiki/Algarve_Internat...,Portimão,Portugal
...,...,...,...,...
73,TI Circuit Aida,https://en.wikipedia.org/wiki/Okayama_Internat...,Mimasaka,Japan
74,Valencia Street Circuit,https://en.wikipedia.org/wiki/Valencia_Street_...,Valencia,Spain
75,Watkins Glen International,https://en.wikipedia.org/wiki/Watkins_Glen_Int...,Watkins Glen,United States
76,Yas Marina Circuit,https://en.wikipedia.org/wiki/Yas_Marina_Circuit,Abu Dhabi,United Arab Emirates


### Enrichissement géographique

Chaque circuit est identifié par son nom et l’URL de sa page Wikipedia. L’objectif
est d’extraire, depuis l’infobox quand disponible :

- latitude  
- longitude  

Ces variables servent de socle pour la contextualisation géographique
(stations météo, distances, agrégations) sans appliquer de logique métier.

La latitude et la longitude ne sont pas dans la table principale. Chaque page
individuelle est visitée pour récupérer les coordonnées depuis les balises
Wikipedia (`geo-dec`, `latitude`, `longitude`).

Formats disponibles :
- **DMS** (`geo-dms`)  
- **Décimal** (`geo-dec`)

Le format décimal est privilégié pour son exploitation numérique directe. Si
absent, l’extraction utilise le format DMS et le traite pour stocker le bon format de données.

In [10]:
def extract_coordinates(url, headers):
    """
    Extrait les coordonnées (latitude, longitude) depuis une page Wikipedia.
    
    Paramètres :
        url (str) : URL de la page Wikipedia.
        headers (dict) : En-têtes HTTP à utiliser pour la requête.
        
    Retour :
        (lat, lon) (tuple de float) : Coordonnées décimales, ou (None, None) si non trouvées.
    """
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException:
        return None, None

    soup = BeautifulSoup(response.text, "html.parser")
    
    # 1. Récupération coordonnées décimales cachées via classe geo
    geo = soup.select_one("table.infobox span.geo-inline span.geo-nondefault span.geo")
    if geo:
        try:
            lat_str, lon_str = geo.text.split(";")
            return float(lat_str.strip()), float(lon_str.strip())
        except ValueError:
            return None, None

    # 2. Sinon récupération coordonnées DMS affichées via classe geo-dev + traitement
    geo_dec = soup.select_one("table.infobox span.geo-inline span.geo-default span.geo-dec")
    if geo_dec:
        try:
            lat_txt, lon_txt = geo_dec.text.strip().split()
            def parse_dms(val):
                sign = -1 if val[-1] in ("S", "W") else 1
                return sign * float(val[:-2].replace("°",""))
            return parse_dms(lat_txt), parse_dms(lon_txt)
        except Exception:
            return None, None

    # 3. Pas de coordonnées trouvées
    return None, None

In [11]:
latitudes = []
longitudes = []

for _, row in circuits_df.iterrows():
    url = row["circuit_url"]
    lat, lon = extract_coordinates(url, HEADERS)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(1)

In [12]:
circuits_df["latitude"] = latitudes
circuits_df["longitude"] = longitudes

In [13]:
circuits_df.head(10)

,circuit_name,circuit_url,locality,country,latitude,longitude
0,Adelaide Street Circuit,https://en.wikipedia.org/wiki/Adelaide_Street_...,Adelaide,Australia,-34.93056,138.62056
1,Ain-Diab Circuit,https://en.wikipedia.org/wiki/Ain-Diab_Circuit,Casablanca,Morocco,33.57861,-7.68750
2,Aintree Motor Racing Circuit,https://en.wikipedia.org/wiki/Aintree_Motor_Ra...,Aintree,United Kingdom,53.47694,-2.94056
3,Albert Park Circuit,https://en.wikipedia.org/wiki/Albert_Park_Circuit,Melbourne,Australia,-37.84972,144.96833
4,Algarve International Circuit,https://en.wikipedia.org/wiki/Algarve_Internat...,Portimão,Portugal,37.23194,-8.63194
5,Autódromo do Estoril,https://en.wikipedia.org/wiki/Aut%C3%B3dromo_d...,Estoril,Portugal,38.75083,-9.39417
6,Autódromo Hermanos Rodríguez,https://en.wikipedia.org/wiki/Aut%C3%B3dromo_H...,Mexico City,Mexico,19.40611,-99.09250
7,Autódromo Internacional do Rio de Janeiro,https://en.wikipedia.org/wiki/Aut%C3%B3dromo_I...,Rio de Janeiro,Brazil,-22.97556,-43.39500
8,Autodromo Internazionale del Mugello,https://en.wikipedia.org/wiki/Autodromo_Intern...,Scarperia e San Piero,Italy,43.99750,11.37194
9,Autodromo Internazionale Enzo e Dino Ferrari,https://en.wikipedia.org/wiki/Imola_Circuit,Imola,Italy,44.34111,11.71333


----------------

## 4. Vérification et validation de la table circuits

La table `circuits_df` contient pour chaque circuit :  
- nom officiel  
- URL Wikipedia (traçabilité)  
- ville et pays  
- latitude et longitude  

Avant toute analyse ou ETL, une **validation minimale** est effectuée :  
- vérifier la structure du DataFrame  
- identifier les valeurs manquantes  
- détecter incohérences évidentes  

Aucune correction ou enrichissement n’est appliqué à ce stade.

### Aperçu général

In [15]:
circuits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   circuit_name  78 non-null     object 
 1   circuit_url   78 non-null     object 
 2   locality      78 non-null     object 
 3   country       78 non-null     object 
 4   latitude      77 non-null     float64
 5   longitude     77 non-null     float64
dtypes: float64(2), object(4)
memory usage: 3.8+ KB


### Vérification des valeurs manquantes

### Rôle des coordonnées géographiques

Latitude et longitude sont essentielles pour la sélection et l’alignement des
stations météo en couche Silver.

En **Bronze** :
- les valeurs manquantes sont **acceptées**  
- elles doivent être **documentées**  

L’objectif est de qualifier la complétude des données, sans les corriger.

In [16]:
circuits_df.isna().sum().sort_values(ascending=False)

latitude        1
longitude       1
circuit_url     0
circuit_name    0
country         0
locality        0
dtype: int64

In [17]:
circuits_df[circuits_df["latitude"].isna() | circuits_df["longitude"].isna()]

,circuit_name,circuit_url,locality,country,latitude,longitude
19,Caesars Palace Grand Prix Circuit,https://en.wikipedia.org/wiki/Caesars_Palace_G...,Paradise,United States,NaN,NaN


Nous pouvons voir qu'il nous manque les coordonnées pour le circuit du Caesars Palace. Après vérification sur sa page Wikipédia, nous remarquons que ses latitudes et longitudes ne sont pas renseignées, il ne s'agit donc pas d'une erreur dans le code.  

Ce circuit nécessitera une décision ultérieure :  
- exclusion,
- enrichissement manuelle,
- ou fallback géographique  

Les autres données semblent être bien renseignées.

### Contrôle de cohérence géographique

Avant d'utiliser ces coordonnées comme référence géographique, il faut vérifier qu'elles soient plausibles, cohérentes, et non erronnées.  

Les valeurs doivent respecter ces bornes, sinon il s'agirait d'une erreur de scraping, de parsing, ou même de renseignement :  
- Latitude ∈ [-90 ; 90]
- Longitude ∈ [-180 ; 180]

In [18]:
invalid_coords = circuits_df[
    (circuits_df["latitude"].abs() > 90) |
    (circuits_df["longitude"].abs() > 180)
]

invalid_coords

,circuit_name,circuit_url,locality,country,latitude,longitude


Les coordonnées semblent ne montrer aucune incohérences.

### Vérification des doublons logiques

Chaque circuit correspond à une localisation unique.  
Une duplication stricte des coordonnées peut indiquer :  
- changement de nom dans le temps  
- redondance de pages Wikipedia  
- ambiguïté historique  

Ces cas ne sont **pas traités en Bronze** et seront gérés, si nécessaire, dans
les couches supérieures.

In [19]:
circuits_df.duplicated(
    subset=["latitude", "longitude"],
    keep=False
).sum()

np.int64(0)

--------------

## 5. Rôle des données

### Rôle des circuits

Les circuits ne sont pas des features prédictives directes, mais servent de
**référence géographique** pour :  
- contextualiser les données météo  
- sélectionner les stations pertinentes  
- assurer la cohérence spatiale en Silver et Gold  

La qualité de cette table impacte indirectement la performance du modèle final.

### Latitude et longitude : variables structurelles

Ces variables ne servent pas de features explicatives. Elles permettent uniquement :  
- d’ancrer chaque circuit dans l’espace  
- de préparer l’alignement avec Meteostat  
- d’assurer la traçabilité géographique  

---------

## 6. Décisions ETL pour la couche Bronze, et transition vers la couche Silver

Les décisions suivantes sont appliquées dans ce notebook :

- scraping exclusivement depuis Wikipedia,
- conservation des données brutes,
- absence de correction manuelle,
- acceptation des valeurs manquantes,
- traçabilité via l’URL source.

Ce notebook ne réalise :
- aucune jointure,
- aucun feature engineering,
- aucun calcul métier.

La table `circuits` sert de référence pour :  
- l’alignement avec Meteostat  
- la sélection des stations météo  
- l’enrichissement des données OpenF1  

Aucune logique d’alignement spatial ou temporel n’est appliquée en Bronze.

--------------

## 7. Sauvegarde de la table circuits

La table `circuits` est sauvegardée pour :  
- figer l’état des données Wikipedia  
- garantir la reproductibilité  
- éviter le re-scraping  
- séparer exploration (Bronze) et enrichissement (Silver)  

Détails de la sauvegarde :  
- format CSV, lisible et universel  
- sans index  
- sans nettoyage (traitement en Silver)

In [21]:
BRONZE_DIR = Path("../data/bronze")
BRONZE_DIR.mkdir(parents=True, exist_ok=True)

output_path = BRONZE_DIR / "circuits_wikipedia_bronze.csv"

# Sauvegarde sans index pour préserver un format tabulaire propre
circuits_df.to_csv(output_path, index=False)

print(f"Table circuits sauvegardée en couche Bronze : {output_path}")

Table circuits sauvegardée en couche Bronze : ..\data\bronze\circuits_wikipedia_bronze.csv


------------

## 8. Conclusion — Wikipedia Circuits Exploration

Ce notebook a exploré et structuré les informations sur les circuits de F1 à
partir de Wikipedia.  

Étapes réalisées :  
- identification des circuits via la page de référence  
- extraction des informations structurantes (nom, pays, localité)  
- enrichissement géographique depuis les pages individuelles  
- construction de la table de référence `circuits`  
- évaluation de la qualité et des limites des données  

Le travail est strictement **exploratoire et préparatoire**, sans interaction
avec les autres sources.

### Positionnement dans l’architecture Bronze / Silver / Gold

La table `circuits` est une **table Bronze** :  
- données brutes issues du scraping  
- pas de correction manuelle  
- valeurs manquantes conservées  
- traçabilité via les URL Wikipedia  

Elle sert de référence géographique pour un enrichissement ultérieur en Silver.

### Décision d’extraction et persistance des données

La table est exportée pour :  
- isoler clairement la couche Bronze  
- éviter le re-scraping  
- figer l’état des données  
- faciliter la réutilisation dans les notebooks suivants  

Format choisi : CSV, simple, tabulaire, sans transformation métier, compatible
avec Python.

### Continuité avec les couches Silver et Gold

Les étapes suivantes du projet seront :  
- alignement avec les données météo  
- sélection des stations Meteostat  
- enrichissement des données OpenF1 pour modélisation  

Ces traitements relèvent des couches Silver et Gold et ne sont pas réalisés ici.